In [ ]:
def mean_reverse(df, lag = 1, long_short_market = False, stock_outliers = False):
    # Transform data from long to wide format and change returns from log to regular
    r = df.pivot(index = 'd', columns = 'id', values = 'r')
    R = r.apply(lambda x: np.exp(x) - 1)
    
    # Calculate weights for dollar-neutral portfolio
    def weights_no_lag(day): 
        day_d = day - day.mean()
        c = -1 / (day_d.abs().sum()/2)
        return c * day_d
    
    # Assign weights for given lag
    w = R.apply(weights_no_lag, axis=1).shift(lag)
    w_long = pd.melt(w.reset_index(), id_vars=['d'], value_name='w')
    w_long['k'] = lag
    
    # Calculate returns of portfolio
    P_r = (w * R).sum(axis=1, skipna=False)
    
    # Calculate characteristics of the strategy
    mean_r = P_r.mean() * 252
    volatility = P_r.std() * math.sqrt(252)
    sharpe = mean_r / volatility
    
    # If we want extra information (performance of long, short and market for Q1), 
    # we will make additional operations:
    if long_short_market == True:
        L_r = (w[w > 0] * R).sum(axis=1)
        S_r = (w[w < 0] * R).sum(axis=1)
        M_r = R.apply(np.mean, axis=1)
        
        return [mean_r, volatility, sharpe], w_long, P_r, L_r, S_r, M_r
    
    elif stock_outliers == True:
        stock_returns = (w * R).sum(axis=0, skipna=True) 
        return [mean_r, volatility, sharpe], w_long, P_r, stock_returns
    
    else:
        return [mean_r, volatility, sharpe], w_long, P_r